# Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# YouTube Reporting API: Bulk Reporting Job Creation for Content Owners

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YouTubeLabs/code-samples/blob/main/yt_bulk_reporting_job_creation_for_co/yt_bulk_reporting_job_creation_for_co.ipynb)

## Purpose of this notebook

The 
[YouTube Reporting API](https://developers.google.com/youtube/reporting/v1/reference/rest) 
enables developers to schedule reporting jobs and then download generated bulk 
reports. The API supports a predefined set of reports, each of which contains a 
comprehensive set of YouTube Analytics data for a content owner.

You can run this Colab notebook to streamline your YouTube Reporting API 
workflow by automating the creation of all available 
[reporting jobs](https://developers.google.com/youtube/reporting/v1/reference/rest) 
in bulk. This ensures you consistently collect comprehensive YouTube Analytics 
data for your content owner, without manually setting up each report.

For an example on how you can subsequentially download the retrieved reports, 
you can refer to 
[this other Colab notebook](https://github.com/YouTubeLabs/code-samples/blob/main/authentication_and_report_bulk_download/authentication_and_report_bulk_download.ipynb).

Please note that this notebook creates reporting jobs for **ALL** non-system 
managed reports that you are eligible to receive but are not yet receiving,
excluding the report types that are marked as deprecated. If you have in place a
workflow that automatically downloads reports for all existing jobs, running
this script could significantly increase the volume of reports that you
download.

## Requirements

In order to run this Colab you will need the following information:
1. Your Content Owner ID (a 22-character string found in your YouTube Studio 
Content Owner URL).

2. A service account key that has access to your CMS

    a. [Create a Google Cloud Project](https://support.google.com/googleapi/answer/6251787?sjid=491706405724629806-NC#zippy=%2Ccreate-a-project) if you don't have one.

    b. Enable the YouTube Reporting APIs on of the 
    [Google Cloud Console](https://console.cloud.google.com/apis/library/youtubereporting.googleapis.com).

    c. Create a service account in IAM section and a service account key for it 
    using the 
    [following tutorial](https://cloud.google.com/iam/docs/keys-create-delete).

    d. Add your service account as a user in your CMS using the 
    [following tutorial](https://support.google.com/youtube/answer/4524878?hl=en). 
    The email adress to invite should be the one associated with your service 
    account ending with iam.gserviceaccount.com. The service account will 
    automatically accept the invite sent from the CMS.\
    If you plan on downloading reports with revenue numbers, you will need to 
    have permission to view revenue for the content owner.

## Important note on credential management

This Colab notebook, like others in this repository, uses a service account JSON 
key for authentication. While convenient for educational and demonstration 
purposes in a Colab notebook, be sure to handle such a file, which contains 
sensitive credentials, with extreme care.

Read [here](https://developers.google.com/youtube/reporting/guides/authorization) 
for more information on authorization flows.

Please enter your content owner ID in the field below
on the right.

In [ ]:
import json
import time
from google.colab import files
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Content Owner ID: Replace with your actual 22-character Content Owner ID.
# This can be found in the URL to your content owner in YouTube Studio.
CONTENT_OWNER_ID = ""  # @param {type:"string"}

if not CONTENT_OWNER_ID:
  raise ValueError(
      "Error: CONTENT_OWNER_ID cannot be empty. "
      "Please provide your Content Owner ID."
  )

# Authentication Scopes for YouTube Reporting API
SCOPES = ["https://www.googleapis.com/auth/yt-analytics-monetary.readonly"]
API_SERVICE_NAME = "youtubereporting"
API_VERSION = "v1"

## Authentication

The function below will authenticate you to the reporting API.

The authentication process gives rights to the following scope:
yt-analytics-monetary.readonly

In [ ]:
def get_authenticated_service():
  """Authenticates to the YouTube Reporting API using a service account key.

  This function will prompt you to upload your service account JSON file.

  Returns:
      A YouTube Reporting API service object, or None if authentication fails.
  """
  print("Please upload your service account key JSON file.")
  service_account_upload = files.upload()
  try:
    service_account_json = json.loads(
        next(iter(service_account_upload.values()))
    )
  except StopIteration:
    print("Error: No file was uploaded.")
    return None

  credentials = service_account.Credentials.from_service_account_info(
      service_account_json, scopes=SCOPES
  )
  print("Authentication successful.")
  return build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

## List the report types

This function retrieves all available report types, explicitly excluding
system-managed reports since they do not require manual job creation and report
types that are not flagged as deprecated (as per documentation here:
https://developers.google.com/youtube/reporting/v1/reference/rest/v1/jobs/create#errors).
An exhaustive list of bulk reports supported by the API can be found 
[here](https://developers.google.com/youtube/reporting/v1/reports/content_owner_reports).

In [ ]:
def list_report_types(youtube_reporting, **kwargs):
  """Returns all available report types from the YouTube Reporting API.

  Excludes system-managed reports.

  Args:
      youtube_reporting: The authenticated YouTube Reporting API service.
      **kwargs: Additional keyword arguments for the API request.

  Returns:
      A list of available report type dictionaries.
  """
  non_system_managed_report_types = []
  try:
    request = youtube_reporting.reportTypes().list(
        includeSystemManaged=False, **kwargs
    )
    while request:
      response = request.execute()
      if "reportTypes" in response:
        for report_type in response["reportTypes"]:
          # Check if 'deprecateTime' is present
          if "deprecateTime" in report_type:
            print(f"Skipping deprecated report type: {report_type.get('id', 'Unknown')}")
          else:
            non_system_managed_report_types.extend([report_type])
      request = youtube_reporting.reportTypes().list_next(request, response)

  except HttpError as e:
    print(
        f"An HTTP error {e.resp.status} occurred while listing report types:\n"
        f"{e.content}"
    )
    return []
  return non_system_managed_report_types

## List the existing reporting jobs

This function retrieves all previously created reporting jobs. Note that jobs
for system-managed reports are automatically generated by YouTube and are
therefore ignored by this function.

In [ ]:
def list_reporting_jobs(youtube_reporting, **kwargs):
  """Returns all existing reporting jobs.

  Excludes jobs for system-managed reports.

  Args:
      youtube_reporting: The authenticated YouTube Reporting API service.
      **kwargs: Additional keyword arguments for the API request.

  Returns:
      A list of existing reporting job dictionaries.
  """
  jobs = []
  try:
    request = youtube_reporting.jobs().list(
        includeSystemManaged=False, **kwargs
    )
    while request:
      response = request.execute()
      if "jobs" in response:
        jobs.extend(response["jobs"])
      request = youtube_reporting.jobs().list_next(request, response)

  except HttpError as e:
    print(
        f"An HTTP error {e.resp.status} occurred while listing jobs:\n"
        f"{e.content}"
    )
    return []
  return jobs

## Create missing reporting jobs

The function below creates reporting jobs for all the report types (excluding
system-managed reports) that don't have one yet.

The function applies the name  `"Job bulk created via script for {report_type_id}"`
to the job. Update the code below accordingly if you desire a different name to be applied.


In [ ]:
def create_reporting_job(youtube_reporting, report_type_id, content_owner_id):
  """Creates a new reporting job for a given report type ID.

  Args:
      youtube_reporting: The authenticated YouTube Reporting API service.
      report_type_id: The ID of the report type to create a job for.
      content_owner_id: The ID of the content owner.

  Returns:
      The created job dictionary, or None if job creation fails.
  """
  body = {
      "reportTypeId": report_type_id,
      "name": f"Job bulk created via script for {report_type_id}",
  }
  try:
    job = (
        youtube_reporting.jobs()
        .create(onBehalfOfContentOwner=content_owner_id, body=body)
        .execute()
    )
    print(
        f"Successfully created job '{job['id']}' "
        f"for report type '{report_type_id}'"
    )
    return job
  except HttpError as e:
    print(
        f"An HTTP error {e.resp.status} occurred while creating job "
        f"for {report_type_id}:\n{e.content}"
    )
    return None

## Run the script

In [ ]:
# The code below will authenticate you to the reporting API.
youtube_reporting = get_authenticated_service()

In [ ]:
# Main script to identify and create the missing jobs.

if youtube_reporting:
  available_report_types = list_report_types(
      youtube_reporting, onBehalfOfContentOwner=CONTENT_OWNER_ID
  )
  print(
      f"Found {len(available_report_types)} available report types "
      "(excluding system-managed reports)."
  )
  existing_jobs = list_reporting_jobs(
      youtube_reporting, onBehalfOfContentOwner=CONTENT_OWNER_ID
  )
  print(
      f"Found {len(existing_jobs)} total existing reporting jobs "
      "(excluding auto-generated jobs for system-managed reports)."
  )

  existing_job_report_type_ids = {
      job["reportTypeId"] for job in existing_jobs if "reportTypeId" in job
  }

  new_jobs_created_count = 0
  for report_type in available_report_types:
    if report_type["id"] not in existing_job_report_type_ids:
      created_job = create_reporting_job(
          youtube_reporting, report_type["id"], CONTENT_OWNER_ID
      )
      if created_job:
        new_jobs_created_count += 1
        time.sleep(0.5)

  print(f"Total new reporting jobs created: {new_jobs_created_count}")
else:
  print("Authentication failed.")